In [1]:
import numpy as np
import pandas as pd
import pennylane as qml
from susy_qm import calculate_Hamiltonian
from qiskit.quantum_info import SparsePauliOp
import itertools

from qutip import Qobj, concurrence, ptrace

import matplotlib.pyplot as plt

In [117]:
potential = 'QHO'
cutoff = 4

In [118]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

min_3_ev = eigenvalues.argsort()[:3]

hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [119]:
min_eigenvector.reshape(2,cutoff)

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [120]:
hamiltonian

SparsePauliOp(['III', 'IZI', 'IZZ', 'ZII'],
              coeffs=[ 1.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j])

In [121]:
hamiltonian = hamiltonian.to_list()
#hamiltonian.append(('YII', (-1.0+0j)))

In [122]:
hamiltonian

[('III', (1.5+0j)),
 ('IZI', (-0.5000000000000001+0j)),
 ('IZZ', (-0.5000000000000001+0j)),
 ('ZII', (0.5+0j))]

In [123]:
single_q_h = []
total_e = 0
for pauli_str, c in hamiltonian:
    
    qubits_involved = [idx for idx, op in enumerate(pauli_str) if op != 'I']
    num_qubits = len(qubits_involved)

    if num_qubits == 1:
        single_q_h.append((pauli_str, c))
             
    op = SparsePauliOp(pauli_str, c)
    op_matrix = op.to_matrix()
    exp_val = np.vdot(min_eigenvector, op_matrix.dot(min_eigenvector))
    total_e += exp_val
    print("Expectation value for term", pauli_str, ":", exp_val)


Expectation value for term III : (1.5+0j)
Expectation value for term IZI : (-0.5000000000000001+0j)
Expectation value for term IZZ : (-0.5000000000000001+0j)
Expectation value for term ZII : (-0.5+0j)


In [114]:
total_e

np.complex128(2.7755575615628914e-16+0j)

In [ ]:
import numpy as np
from qiskit.quantum_info.operators import SparsePauliOp

# Define your SparsePauliOp (using your provided pauli strings and coefficients)
paulis = ['III', 'IIX', 'IXI', 'IXX', 'IXZ', 'IYY', 'IZI', 'IZX', 'IZZ', 'ZII', 'ZIX', 'ZXX', 'ZYY', 'ZZX']
coeffs = [5.375, 3.027187, 2.41481457, 2.4330127,
          -0.64704761, 1.5669873, -1.75, -1.25942005,
          -1.75, 0.5, 0.96592583, 0.5, 0.5, -0.25881905]
H_sparse = SparsePauliOp(paulis, coeffs=np.array(coeffs, dtype=complex))

# Suppose you also have your eigenstate (as a numpy array of shape (8,))
psi = np.array([0, 0, 0, 0, 0.96698015, -0.16164595, -0.18805279, 0.05878883], dtype=complex)

# Calculate the expectation value for each term and group contributions:
local_energy = [0, 0, 0]  # one for each qubit (0,1,2)
for pauli_str, c in zip(paulis, coeffs):
    # Identify indices of non-identity operators:
    qubits_involved = [idx for idx, op in enumerate(pauli_str) if op != 'I']
    
    # Compute expectation value for this term:
    # (Here, you would need to build the full operator corresponding to pauli_str,
    #  then compute <psi| (c*operator) |psi>.)
    # For illustration, assume exp_val holds the computed expectation value:
    exp_val = ...  # compute expectation value for this term

    # Distribute the contribution:
    if len(qubits_involved) == 1:
        # All of the energy goes to that qubit
        local_energy[qubits_involved[0]] += exp_val
    else:
        # For interaction terms, split equally (or use another convention)
        split_val = exp_val / len(qubits_involved)
        for q in qubits_involved:
            local_energy[q] += split_val

print("Local energy contributions per qubit:", local_energy)
